# Generative AI POC – Exploración de Datos IFC

Este notebook forma parte de una Prueba de Concepto (POC) cuyo objetivo es
evaluar el uso de Inteligencia Artificial Generativa (GenAI) para comprender
datos técnicos de proyectos inmobiliarios.

A partir de un archivo IFC de ejemplo, se extrae información estructurada
del proyecto (espacios, muros, puertas, ventanas) que luego puede ser
consultada mediante lenguaje natural usando modelos de GenAI.

Este notebook **no representa una solución productiva**, sino un artefacto
demostrativo para un workshop de Applied Generative AI, orientado a validar
la viabilidad técnica del enfoque.

**Nota de alcance (CRISP-DM):**
- **Etapa 1 (Business Understanding):** documentada en el repositorio (documento de la POC / CRISP-DM).
- **Este notebook se enfoca en Etapa 2 (Data Understanding)** y **Etapa 3 (Data Preparation)** a nivel mínimo (ingesta, inventario, limpieza y métricas controladas).
- **Etapa 4 (Modeling / Vertex AI):** se plantea como siguiente paso y se implementará después de la aprobación del documento de la POC.



---

### Flujo general del POC

Archivo IFC
   │
   ▼
Ingesta y Validación
   │
   ▼
Exploración de Entidades (IfcSpace, IfcWall, IfcDoor, IfcWindow)
   │
   ▼
Limpieza y Métricas Derivadas
   │
   ▼
Contexto Estructurado (JSON)
   │
   ▼
Resumen Textual
   │
   ▼
Interacción con GenAI (Prompt + Q&A)


Nota: Este notebook está diseñado para leerse de forma secuencial, siguiendo una narrativa técnica desde la ingesta del dato hasta la preparación del contexto para modelos de Inteligencia Artificial Generativa.

## Suposiciones y decisiones de diseño

Para esta Prueba de Concepto se tomaron las siguientes decisiones de diseño:

- Se trabaja exclusivamente con un archivo IFC de ejemplo (datos sintéticos, no productivos).
- No se realiza inferencia semántica avanzada (ej. tipo de espacio, uso, categoría funcional).
- Las métricas derivadas se limitan a relaciones simples (conteos y promedios) para evitar conclusiones no sustentadas.
- El contexto generado está pensado para consumo por modelos de GenAI sin entrenamiento específico en dominio BIM.
- No se implementa interfaz de usuario ni persistencia en base de datos en esta fase.

Estas decisiones son intencionales y buscan mantener el POC simple, explicable y controlado.


## Riesgos y limitaciones técnicas

Como toda Prueba de Concepto, este ejercicio presenta riesgos y limitaciones que deben tenerse en cuenta:

- **Calidad del IFC**: Si el archivo de origen tiene nombres inconsistentes, valores nulos o mala estructura, el contexto generado puede ser incompleto o engañoso.
- **Generalización**: Los resultados observados con este archivo de ejemplo no garantizan el mismo comportamiento en modelos IFC de otros proyectos o proveedores.
- **Escalabilidad**: Este flujo no está optimizado para archivos IFC de gran tamaño o proyectos con miles de entidades.
- **Dependencia del modelo GenAI**: La calidad de las respuestas depende directamente de la capacidad del modelo para interpretar el contexto proporcionado.
- **Ausencia de validación semántica**: No se valida que los espacios o elementos tengan un uso correcto desde el punto de vista arquitectónico o normativo.

Estos riesgos son aceptables en el contexto de un POC, pero deben abordarse antes de una implementación productiva.


## Criterios de éxito de la Prueba de Concepto

Se considera que esta Prueba de Concepto es exitosa si se cumplen los siguientes criterios:

- El archivo IFC puede ser cargado y procesado sin errores.
- Se obtiene un inventario consistente de entidades principales (espacios, muros, puertas, ventanas).
- Se genera un contexto estructurado (JSON) a partir de los datos extraídos.
- Se produce un resumen textual comprensible del proyecto inmobiliario.
- Un modelo de GenAI es capaz de responder preguntas coherentes basándose únicamente en el contexto proporcionado.
- No se generan inferencias falsas ni se “alucinan” datos inexistentes.

El objetivo no es la precisión arquitectónica, sino la **viabilidad técnica del enfoque**.


## Trazabilidad con metodología CRISP-DM

| Etapa CRISP-DM        | Implementación en este notebook |
|----------------------|----------------------------------|
| Business Understanding | Definición del objetivo del POC y alcance no productivo |
| Data Understanding     | Inventario de entidades IFC, exploración de espacios y conteos |
| Data Preparation       | Limpieza de nombres, métricas derivadas, contexto estructurado |
| Modeling               | No implementado en esta fase (planificado con Vertex AI) |
| Evaluation              | Validación manual mediante preguntas y respuestas |
| Deployment              | Fuera de alcance para este POC |

Esta trazabilidad asegura alineación metodológica y control de alcance.


## 1. Carga del archivo IFC

En esta sección se carga un archivo IFC de ejemplo y se inicializa el modelo
utilizando `ifcopenshell`.


## 1.1 Validación e Ingesta del archivo IFC

Antes de procesar el archivo IFC, se realiza una validación mínima para asegurar:
- Existencia del archivo
- Extensión correcta (.ifc)
- Trazabilidad mediante hash
- Tamaño del archivo

Esto garantiza una ingesta controlada y reproducible.


In [1]:
from pathlib import Path
import os
import hashlib
import json

IFC_PATH = Path("../data/sample.ifc")

def sha256_file(path: Path) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(8192), b""):
            h.update(chunk)
    return h.hexdigest()

# Validación mínima (ingesta)
assert IFC_PATH.exists(), f"No existe el archivo IFC en: {IFC_PATH.resolve()}"
assert IFC_PATH.suffix.lower() == ".ifc", f"Extensión inválida: {IFC_PATH.suffix}"

file_size_mb = IFC_PATH.stat().st_size / (1024 * 1024)
print(f"OK: Archivo encontrado -> {IFC_PATH}")
print(f"Tamaño: {file_size_mb:.2f} MB")
print(f"SHA256: {sha256_file(IFC_PATH)[:16]}... (truncado)")


OK: Archivo encontrado -> ..\data\sample.ifc
Tamaño: 2.27 MB
SHA256: b347a2c8aa8fff6d... (truncado)


In [2]:
import ifcopenshell

try:
    model = ifcopenshell.open(str(IFC_PATH))
    print("Modelo cargado correctamente")
except Exception as e:
    raise RuntimeError(f"Error cargando IFC: {IFC_PATH} -> {e}")



Modelo cargado correctamente


## 2. Inventario inicial del modelo IFC

Se realiza un inventario básico de entidades para comprender la estructura general del proyecto y validar la presencia de los elementos relevantes para la POC.


In [3]:
# Inventario mínimo (Data Understanding)
entity_types = ["IfcProject", "IfcBuilding", "IfcBuildingStorey", "IfcSpace", "IfcWall", "IfcDoor", "IfcWindow"]

inventory = {}
for t in entity_types:
    inventory[t] = len(model.by_type(t))

print("Inventario (conteos por entidad):")
for k, v in inventory.items():
    print(f"- {k}: {v}")


Inventario (conteos por entidad):
- IfcProject: 1
- IfcBuilding: 1
- IfcBuildingStorey: 4
- IfcSpace: 21
- IfcWall: 57
- IfcDoor: 14
- IfcWindow: 24


## 2.1 Exploración de datos del proyecto

A continuación se exploran los elementos principales del modelo IFC
para identificar espacios y componentes relevantes del proyecto.


## 2.2 Exploración de espacios (IfcSpace)

Se analizan los espacios del proyecto para:
- Identificar ambientes disponibles
- Evaluar calidad de nombres
- Detectar valores nulos y duplicados


In [4]:
spaces = model.by_type("IfcSpace")

def clean_name(x):
    if not x:
        return None
    x = str(x).strip()
    x = " ".join(x.split())  # quita dobles espacios
    return x if x else None

raw_names = [(s.LongName or s.Name) for s in spaces]
clean_names = [clean_name(n) for n in raw_names]

missing_name_count = sum(1 for n in clean_names if n is None)
unique_names = []
seen = set()
for n in clean_names:
    if n and n not in seen:
        unique_names.append(n)
        seen.add(n)

print(f"Número total de espacios encontrados: {len(spaces)}")
print(f"Espacios sin nombre (nulos/vacíos): {missing_name_count}")
print(f"Nombres únicos de espacios: {len(unique_names)}")

print("\nPrimeros 10 nombres (únicos):")
for i, name in enumerate(unique_names[:10], start=1):
    print(f"{i}. {name}")



Número total de espacios encontrados: 21
Espacios sin nombre (nulos/vacíos): 0
Nombres únicos de espacios: 12

Primeros 10 nombres (únicos):
1. Foyer
2. Hallway
3. Bathroom 1
4. Utility
5. Stair
6. Room
7. Roof
8. Living Room
9. Kitchen
10. Bathroom 2


## 3. Enriquecimiento básico de datos

Se calculan métricas derivadas simples a partir de los conteos extraídos del IFC, con el fin de enriquecer el contexto sin introducir inferencias no sustentadas.


In [5]:
walls = model.by_type("IfcWall")
doors = model.by_type("IfcDoor")
windows = model.by_type("IfcWindow")

# Métricas derivadas simples (enriquecimiento controlado)
spaces_count = len(spaces)
walls_count = len(walls)
doors_count = len(doors)
windows_count = len(windows)

walls_per_space = (walls_count / spaces_count) if spaces_count else None
windows_per_space = (windows_count / spaces_count) if spaces_count else None

derived_metrics = {
    "walls_per_space": round(walls_per_space, 2) if walls_per_space is not None else None,
    "windows_per_space": round(windows_per_space, 2) if windows_per_space is not None else None,
}

print("Métricas derivadas (enriquecimiento básico):")
for k, v in derived_metrics.items():
    print(f"- {k}: {v}")



Métricas derivadas (enriquecimiento básico):
- walls_per_space: 2.71
- windows_per_space: 1.14


In [6]:
walls = model.by_type("IfcWall")
doors = model.by_type("IfcDoor")
windows = model.by_type("IfcWindow")

print(f"Muros: {len(walls)}")
print(f"Puertas: {len(doors)}")
print(f"Ventanas: {len(windows)}")


Muros: 57
Puertas: 14
Ventanas: 24


## 3.1 Transformación a contexto estructurado

Los datos técnicos se transforman a un objeto estructurado que será utilizado como contexto para modelos de Generative AI.


## 4. Resumen del proyecto para consultas con GenAI

En esta sección se genera un resumen textual del proyecto inmobiliario
a partir de los datos extraídos del archivo IFC.

Este resumen será utilizado como **entrada (contexto)** para modelos
de Inteligencia Artificial Generativa, permitiendo responder preguntas
en lenguaje natural sobre el proyecto.


In [7]:
project_context = {
    "source": {
        "ifc_path": str(IFC_PATH),
        "note": "Archivo IFC de ejemplo (no productivo)."
    },
    "counts": {
        "spaces": spaces_count,
        "walls": walls_count,
        "doors": doors_count,
        "windows": windows_count
    },
    "data_quality": {
        "spaces_missing_name": missing_name_count,
        "unique_space_names": len(unique_names)
    },
    "derived_metrics": derived_metrics
}

project_summary_text = (
    "Resumen del proyecto inmobiliario:\n\n"
    f"- Número total de espacios: {spaces_count}\n"
    f"- Número de muros: {walls_count}\n"
    f"- Número de puertas: {doors_count}\n"
    f"- Número de ventanas: {windows_count}\n\n"
    "Este resumen fue generado automáticamente a partir de un archivo IFC,\n"
    "con el objetivo de permitir consultas en lenguaje natural usando GenAI.\n"
)

print(project_summary_text)



Resumen del proyecto inmobiliario:

- Número total de espacios: 21
- Número de muros: 57
- Número de puertas: 14
- Número de ventanas: 24

Este resumen fue generado automáticamente a partir de un archivo IFC,
con el objetivo de permitir consultas en lenguaje natural usando GenAI.



## 5. Control de calidad de datos (mínimo)

En esta POC se reportan métricas básicas de calidad para asegurar trazabilidad y evitar inferencias:

- Completitud: espacios con nombre vs espacios sin nombre.
- Consistencia: conteos por entidad (IfcSpace, IfcWall, IfcDoor, IfcWindow).
- Duplicidad: número de nombres únicos de espacios vs total de espacios.


In [8]:
quality_report = {
    "completeness": {
        "spaces_total": spaces_count,
        "spaces_with_name": spaces_count - missing_name_count,
        "spaces_missing_name": missing_name_count,
        "pct_spaces_with_name": round(((spaces_count - missing_name_count) / spaces_count) * 100, 2) if spaces_count else None
    },
    "duplication": {
        "unique_space_names": len(unique_names),
        "total_spaces": spaces_count
    },
    "counts": project_context["counts"]
}

print("Reporte mínimo de calidad:")
print(json.dumps(quality_report, indent=2, ensure_ascii=False))


Reporte mínimo de calidad:
{
  "completeness": {
    "spaces_total": 21,
    "spaces_with_name": 21,
    "spaces_missing_name": 0,
    "pct_spaces_with_name": 100.0
  },
  "duplication": {
    "unique_space_names": 12,
    "total_spaces": 21
  },
  "counts": {
    "spaces": 21,
    "walls": 57,
    "doors": 14,
    "windows": 24
  }
}


## 5.1 Exportación de artefactos de datos

Se exportan los artefactos generados (contexto y reporte de calidad) para asegurar trazabilidad y reutilización.


In [9]:
OUT_DIR = Path("../data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

out_json = OUT_DIR / "project_context.json"
with open(out_json, "w", encoding="utf-8") as f:
    json.dump(project_context, f, indent=2, ensure_ascii=False)

out_qc = OUT_DIR / "quality_report.json"
with open(out_qc, "w", encoding="utf-8") as f:
    json.dump(quality_report, f, indent=2, ensure_ascii=False)

print(f"OK: Guardado -> {out_json}")
print(f"OK: Guardado -> {out_qc}")


OK: Guardado -> ..\data\processed\project_context.json
OK: Guardado -> ..\data\processed\quality_report.json


## 6. Interacción con GenAI mediante prompt

En esta sección se define el **prompt base** utilizado para interactuar con un modelo
de Inteligencia Artificial Generativa.

Este prompt combina:

- El **contexto técnico del proyecto** generado automáticamente a partir del archivo IFC.
- Instrucciones explícitas para:
  - Interpretar correctamente la información disponible.
  - Responder en lenguaje natural.
  - No inferir ni inventar datos que no estén presentes en el dataset.

Este diseño permite simular de forma controlada cómo un asistente basado en GenAI
podría responder preguntas sobre un proyecto inmobiliario utilizando únicamente
datos estructurados y preprocesados.

**Contexto generado automáticamente:**
- El resumen textual mostrado abajo proviene del procesamiento del archivo IFC (`project_summary_text`).
- El contexto estructurado está disponible en `project_context` (exportado a JSON).
- Ambos artefactos son resultado directo de las etapas:
  - **Etapa 2: Data Understanding**
  - **Etapa 3: Data Preparation**



**Contexto generado automáticamente:**
- El resumen mostrado abajo proviene del procesamiento del archivo IFC (`project_summary_text`).
- El contexto estructurado está disponible en `project_context` (exportado a JSON).


Eres un asistente experto en proyectos inmobiliarios y construcción.

A continuación se te proporciona información técnica extraída automáticamente
desde un archivo IFC de un proyecto inmobiliario.

Tu tarea es:
- Interpretar esta información técnica
- Responder preguntas en lenguaje natural de forma clara y comprensible
- No inventar datos que no estén presentes en la información proporcionada

Información del proyecto:
---
Resumen del proyecto inmobiliario:

- Número total de espacios: 21
- Número de muros: 57
- Número de puertas: 14
- Número de ventanas: 24

Este resumen fue generado automáticamente a partir de un archivo IFC,
con el objetivo de permitir consultas en lenguaje natural usando GenAI.
---

Instrucciones adicionales:
- Si una pregunta no puede responderse con la información disponible, indícalo claramente.
- Usa un lenguaje claro, orientado a usuarios no técnicos.
- Prioriza explicaciones concisas pero completas.



## 6.1 Preguntas de ejemplo para la demostración (alineadas con datos, métricas y alcance)

Las siguientes preguntas están diseñadas para ser utilizadas durante la demostración del POC, utilizando **exclusivamente el contexto generado en este notebook** (conteos, limpieza, métricas derivadas y resumen estructurado).

Están alineadas con:
- Etapa 2: Data Understanding
- Etapa 3: Data Preparation
- Alcance controlado del POC
- Limitaciones documentadas (sin inferencias no sustentadas)

---

### 1. Comprensión general del proyecto (nivel ejecutivo)

> **¿Puedes darme un resumen general del proyecto inmobiliario y sus principales características?**

**Objetivo de la pregunta:**  
Validar que el modelo comprende y sintetiza correctamente:
- Número de espacios
- Cantidad de muros, puertas y ventanas
- Naturaleza general del proyecto

---

### 2. Perspectiva comercial (nivel negocio / ventas)

> **Con la información disponible, ¿qué aspectos del proyecto podrían ser relevantes para un equipo comercial?**

**Objetivo de la pregunta:**  
Demostrar que, aun con datos técnicos limitados, se pueden extraer insights como:
- Cantidad de ambientes
- Complejidad aparente
- Posible atractivo comercial (sin inventar usos ni precios)

---

### 3. Lectura técnica de alto nivel (nivel arquitecto / ingeniería conceptual)

> **¿Qué nivel de complejidad constructiva se puede inferir a partir de los datos del proyecto?**

**Objetivo de la pregunta:**  
Evaluar la capacidad del modelo para:
- Relacionar número de muros, puertas y ventanas
- Comentar complejidad **sin entrar en cálculos estructurales**
- Mantenerse en nivel conceptual

---

### 4. Calidad y consistencia de datos (data mindset)

> **¿Qué puedes decir sobre la calidad de los datos del proyecto y la consistencia de los nombres de los espacios?**

**Objetivo de la pregunta:**  
Conectar directamente con:
- Limpieza de nombres
- Conteo de espacios nulos
- Detección de duplicados
- Métricas de calidad básica

---

### 5. Enriquecimiento controlado (uso de métricas derivadas)

> **¿Qué conclusiones generales se pueden obtener a partir de las métricas derivadas calculadas?**

**Objetivo de la pregunta:**  
Mostrar que el modelo puede:
- Interpretar relaciones simples (ej. muros por espacio, ventanas por espacio)
- Sin caer en inferencias no sustentadas
- Sin inventar usos o tipologías

---

### 6. Control de alcance (pregunta trampa intencional)

> **¿Puedes estimar el costo de construcción o el tipo exacto de uso del proyecto?**

**Objetivo de la pregunta:**  
Demostrar explícitamente que el asistente:
- **Sabe decir “no”**
- Reconoce límites de los datos
- No inventa información
- Respeta las limitaciones del POC

---

Estas preguntas permiten mostrar de forma clara cómo la información técnica del proyecto puede interpretarse desde distintos enfoques:

- Ejecutivo
- Comercial
- Técnico
- Calidad de datos
- Control de inferencia

Todo ello **sin salir del alcance definido** ni comprometer la credibilidad técnica del proyecto.




## 7. Limitaciones del POC (alineadas con alcance y preguntas de demostración)

Esta Prueba de Concepto tiene un alcance **intencionalmente limitado y controlado**, diseñado para validar viabilidad técnica y narrativa de negocio, sin incurrir en complejidad innecesaria ni en riesgos de sobre-promesa.

Las siguientes limitaciones son **conscientes y documentadas**:

### 1. Alcance de datos
- Se trabaja exclusivamente con **un archivo IFC de ejemplo** (datos sintéticos, no productivos).
- No se integran datos reales de clientes, costos, cronogramas, normativas ni información sensible.
- No existe enriquecimiento con fuentes externas (catálogos, GIS, precios, normativas, etc.).

### 2. Nivel de análisis
- El análisis se basa en:
  - Conteo de entidades (IfcSpace, IfcWall, IfcDoor, IfcWindow)
  - Limpieza básica de nombres
  - Métricas derivadas simples (promedios y relaciones)
- **No se realiza análisis geométrico**, estructural, energético ni normativo.
- **No se infiere uso funcional real** (residencial, comercial, mixto) si no está explícitamente presente en los datos.

### 3. Inferencia y GenAI
- El modelo de GenAI **no se entrena ni ajusta** específicamente para dominio BIM.
- La interpretación del proyecto se basa únicamente en el **contexto textual y estructurado generado**.
- El asistente está instruido explícitamente a:
  - No inventar información
  - Indicar cuando algo no puede inferirse con los datos disponibles

### 4. Relación con las preguntas de demostración
Las preguntas de ejemplo incluidas en esta POC están diseñadas para:
- Evaluar comprensión general del proyecto
- Explorar perspectivas ejecutivas, comerciales y técnicas **a nivel conceptual**
- Demostrar control de inferencia y calidad de datos

No están orientadas a:
- Validación técnica de ingeniería
- Cálculo de costos o cronogramas
- Cumplimiento normativo
- Diseño arquitectónico o estructural

### 5. Arquitectura y despliegue
- No se implementa interfaz de usuario.
- No existe persistencia en base de datos.
- No se despliega infraestructura productiva.
- No se integra aún con Vertex AI (Etapa 4 – Modeling pendiente de aprobación).

---

**Estas limitaciones son deliberadas y coherentes con la etapa actual del proyecto (CRISP-DM: Data Understanding & Data Preparation).**  
El objetivo es demostrar **claridad, trazabilidad y control**, no complejidad artificial.


## 7.1 Siguientes pasos (alineados con arquitectura, sin ejecutar Etapa 4)

Los siguientes pasos describen la **evolución natural del POC** hacia una integración con servicios de Generative AI en Google Cloud (Vertex AI), manteniendo el enfoque demostrativo y respetando las restricciones actuales del proyecto.

> **Importante:** Estas acciones **no se ejecutan en esta fase**, ya que la transición a Etapa 4 (Modelado) requiere aprobación de arquitectura. Se documentan como preparación y referencia técnica.

---

### 1. Integración con Vertex AI (conceptual)

- Configurar un proyecto en Google Cloud con Vertex AI habilitado.
- Seleccionar un modelo base (ej. **Gemini Pro**) como motor de Generative AI.
- Utilizar el resumen estructurado generado en este notebook como **contexto de entrada** al modelo.
- Enviar prompts mediante el SDK de Vertex AI desde Python.

**Objetivo:**  
Validar que la arquitectura propuesta es compatible con los flujos ya definidos en este POC.

---

### 2. Estandarización del formato de entrada al modelo

- Definir un esquema estable para el contexto (JSON + texto enriquecido).
- Asegurar que todos los resúmenes del proyecto sigan la misma estructura.
- Versionar el esquema para futuras iteraciones.

**Objetivo:**  
Garantizar consistencia, reproducibilidad y facilidad de mantenimiento.

---

### 3. Refinamiento de prompts por perfil de usuario

Diseñar variantes de prompts para distintos perfiles:

- **Perfil comercial:** enfoque en espacios, iluminación, distribución.
- **Perfil técnico:** enfoque en componentes, densidad, complejidad.
- **Perfil ejecutivo:** enfoque en síntesis y visión general.

**Objetivo:**  
Demostrar que el mismo dataset puede servir a múltiples audiencias.

---

### 4. Automatización del pipeline de preparación de datos

- Integrar el flujo de:
  - Ingesta
  - Limpieza
  - Transformación
  - Enriquecimiento
  - Generación de resumen

- Implementar este flujo como un script o pipeline reutilizable.

**Objetivo:**  
Reducir intervención manual y preparar el camino hacia una solución escalable.

---

### 5. Integración futura con múltiples fuentes de datos

- Evaluar incorporación de datasets sintéticos (ej. Structured3D).
- Diseñar un esquema unificado para combinar IFC + datos semánticos.
- Validar impacto en contexto y latencia.

**Objetivo:**  
Preparar el POC para escenarios más complejos sin comprometer estabilidad.

---

### 6. Preparación para demo de workshop (60–90 min)

- Definir set fijo de preguntas pre-validadas.
- Asegurar tiempos de respuesta aceptables.
- Preparar mensajes de fallback en caso de error.
- Documentar flujo de demo paso a paso.

**Objetivo:**  
Garantizar una demostración fluida, controlada y sin sorpresas técnicas.

---

### 7. Evaluación y métricas

- Definir métricas de:
  - Precisión percibida
  - Relevancia
  - Completitud
  - Tiempo de respuesta

- Registrar observaciones cualitativas de usuarios durante pruebas internas.

**Objetivo:**  
Tener evidencia objetiva para justificar avance a fases posteriores.

---

Estos siguientes pasos están alineados con la metodología CRISP-DM y con las recomendaciones de arquitectura, y sirven como **puente formal entre la preparación de datos (Etapa 3) y el modelado (Etapa 4)**, sin ejecutar aún integración productiva.


## Conclusión

Este notebook valida de forma práctica y controlada la **viabilidad técnica de utilizar Inteligencia Artificial Generativa para la comprensión de datos técnicos de proyectos inmobiliarios** a partir de archivos IFC.

A lo largo del flujo implementado se demostró que:

- Es posible **extraer, limpiar, transformar y estructurar** información técnica compleja desde un modelo IFC.
- Los datos pueden ser convertidos en **contexto textual comprensible** para modelos de lenguaje sin perder fidelidad técnica.
- El mismo conjunto de datos permite responder preguntas desde **múltiples perspectivas**: ejecutiva, comercial y técnica.
- El proceso puede documentarse y controlarse siguiendo una metodología formal (**CRISP-DM**), garantizando trazabilidad y rigor.

Este POC no busca reemplazar herramientas BIM ni sistemas productivos, sino **demostrar valor conceptual y técnico**: reducir fricción en la comprensión de proyectos, habilitar consultas en lenguaje natural y preparar el camino para experiencias más avanzadas (como asistentes virtuales o interfaces conversacionales).

En conclusión, el enfoque propuesto es **técnicamente viable, escalable y alineado con los objetivos del workshop**, y establece una base sólida para evolucionar hacia una integración con servicios de Generative AI en Google Cloud (Vertex AI) en fases posteriores, una vez se cuente con la aprobación de arquitectura correspondiente.
